# Improved Severity Model - Better Confidence & Predictions
This version uses:
- Ensemble methods for better confidence
- Medical keyword features
- Calibrated probabilities
- Better text preprocessing

In [1]:
import pandas as pd
import numpy as np
import re
import joblib
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

In [3]:
# Load training data
train_path = "data/drugLibTrain_raw.tsv"
df_train = pd.read_csv(train_path, sep="\t")

print(f"Loaded {len(df_train)} training records")
df_train = df_train[['sideEffectsReview', 'sideEffects']]
df_train.dropna(inplace=True)
df_train = df_train.reset_index(drop=True)

print(f"After cleaning: {len(df_train)} records")

Loaded 3107 training records
After cleaning: 3032 records


In [5]:
def map_severity_5class(label):
    """Map severity labels to numeric classes"""
    label = label.lower()
    if "no" in label:
        return 0
    elif "mild" in label:
        return 1
    elif "moderate" in label:
        return 2
    elif "extreme" in label:
        return 4
    elif "severe" in label:
        return 3
    else:
        return None

df_train['label'] = df_train['sideEffects'].apply(map_severity_5class)
df_train = df_train[df_train['label'].notna()]
df_train['label'] = df_train['label'].astype(int)

severity_map = {
    0: "No Side Effects",
    1: "Mild",
    2: "Moderate",
    3: "Severe",
    4: "Extremely Severe"
}

print("\nLabel distribution:")
print(df_train['label'].value_counts().sort_index())


Label distribution:
label
0     858
1    1016
2     614
3     369
4     175
Name: count, dtype: int64


In [6]:
# Define medical severity keywords for better feature engineering
SEVERITY_KEYWORDS = {
    'critical': ['died', 'death', 'fatal', 'emergency', 'hospitalization', 'hospitalized', 
                 'icu', 'intensive care', 'life threatening', 'cardiac arrest', 'seizure',
                 'stroke', 'coma', 'organ failure', 'overdose'],
    
    'severe': ['severe', 'extreme', 'unbearable', 'excruciating', 'intense', 'violent',
               'bleeding', 'hemorrhage', 'chest pain', 'heart attack', 'suicidal',
               'paralysis', 'unable to function', 'bedridden', 'collapsed'],
    
    'moderate': ['moderate', 'significant', 'noticeable', 'concerning', 'difficult',
                 'persistent', 'ongoing', 'chronic', 'frequent', 'regular'],
    
    'mild': ['mild', 'slight', 'minor', 'little', 'minimal', 'temporary', 
             'occasional', 'rare', 'tolerable', 'manageable']
}

def extract_severity_features(text):
    """Extract severity-specific features from text"""
    text_lower = text.lower()
    features = {}
    
    for severity, keywords in SEVERITY_KEYWORDS.items():
        # Count how many severity keywords appear
        count = sum(1 for kw in keywords if kw in text_lower)
        features[f'{severity}_keyword_count'] = count
    
    # Additional features
    features['text_length'] = len(text)
    features['word_count'] = len(text.split())
    features['exclamation_count'] = text.count('!')
    features['capitalized_words'] = sum(1 for word in text.split() if word.isupper() and len(word) > 1)
    
    return features

# Extract features for all training samples
severity_features = df_train['sideEffectsReview'].apply(extract_severity_features)
severity_df = pd.DataFrame(severity_features.tolist())

print("\nSeverity features extracted:")
print(severity_df.head())


Severity features extracted:
   critical_keyword_count  severe_keyword_count  moderate_keyword_count  \
0                       0                     0                       0   
1                       0                     0                       0   
2                       0                     1                       0   
3                       0                     0                       0   
4                       0                     1                       0   

   mild_keyword_count  text_length  word_count  exclamation_count  \
0                   0          197          26                  0   
1                   0          258          45                  0   
2                   0           42           6                  0   
3                   1          111          19                  0   
4                   0          150          27                  0   

   capitalized_words  
0                  0  
1                  0  
2                  0  
3           

In [7]:
def clean_text(text):
    """Enhanced text cleaning that preserves medical terms"""
    # Keep important punctuation that indicates severity
    text = text.lower()
    # Remove excessive punctuation but keep sentence structure
    text = re.sub(r'[^a-z\s!.,]', '', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

df_train['clean_text'] = df_train['sideEffectsReview'].apply(clean_text)
print("Sample cleaned text:")
print(df_train['clean_text'].iloc[0])

Sample cleaned text:
cough, hypotension , proteinuria, impotence , renal failure , angina pectoris , tachycardia , eosinophilic pneumonitis, tastes disturbances , anusease anorecia , weakness fatigue insominca weakness


In [8]:
# Create improved TF-IDF vectorizer
vectorizer = TfidfVectorizer(
    max_features=8000,  # Increased for better coverage
    ngram_range=(1, 3),  # Include trigrams for medical phrases
    stop_words='english',
    min_df=2,  # Minimum document frequency
    max_df=0.8,  # Maximum document frequency
    sublinear_tf=True  # Use sublinear term frequency
)

X_tfidf = vectorizer.fit_transform(df_train['clean_text'])
print(f"TF-IDF shape: {X_tfidf.shape}")

# Combine TF-IDF with severity features
X_combined = hstack([X_tfidf, severity_df.values])
y_train = df_train['label'].values

print(f"Combined features shape: {X_combined.shape}")
print(f"Training samples: {len(y_train)}")

TF-IDF shape: (3032, 7173)
Combined features shape: (3032, 7181)
Training samples: 3032


In [9]:
# Train Gradient Boosting Classifier (better for this task)
print("Training Gradient Boosting Classifier...")

gb_model = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10,
    subsample=0.8,
    random_state=42,
    verbose=1
)

# Note: Gradient Boosting works with dense arrays
X_combined_dense = X_combined.toarray()
gb_model.fit(X_combined_dense, y_train)

print("\nModel training complete!")

Training Gradient Boosting Classifier...
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3947           0.0560           24.05m
         2           1.3362           0.0426           24.95m
         3           1.2950           0.0791           26.35m
         4           1.2547           0.0281           26.36m
         5           1.2181           0.0096           28.40m
         6           1.1912           0.0263           29.05m
         7           1.1626           0.0181           30.16m
         8           1.1478           0.0432           29.93m
         9           1.1231           0.0015           28.98m
        10           1.1174           0.0521           28.01m
        20           0.9953          -0.0258           26.33m
        30           0.9296          -0.0066           28.66m
        40           0.8768           0.0057           27.42m
        50           0.8421           0.0342           27.41m
        60           0.8025 

In [10]:
# Calibrate probabilities for better confidence scores
print("\nCalibrating model for better probability estimates...")

calibrated_model = CalibratedClassifierCV(
    gb_model,
    method='isotonic',  # Isotonic regression for calibration
    cv=3
)

calibrated_model.fit(X_combined_dense, y_train)
print("Calibration complete!")


Calibrating model for better probability estimates...
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.3868           0.0557           14.42m
         2           1.3228           0.0363           17.56m
         3           1.2880           0.0814           18.29m
         4           1.2536           0.0591           18.77m
         5           1.2133          -0.0015           19.03m
         6           1.1906           0.0457           19.26m
         7           1.1615           0.0083           19.31m
         8           1.1398           0.0235           19.59m
         9           1.1308           0.0623           19.72m
        10           1.0935          -0.0582           19.78m
        20           0.9607          -0.0061           19.83m
        30           0.8967           0.0201           19.37m
        40           0.8299          -0.0482           18.47m
        50           0.7894          -0.0340           17.64m
        60    

In [11]:
# Evaluate the calibrated model
y_pred = calibrated_model.predict(X_combined_dense)
print("\nTraining Performance (Calibrated Model):")
print(classification_report(y_train, y_pred, target_names=severity_map.values()))

# Check confidence distribution
y_proba = calibrated_model.predict_proba(X_combined_dense)
max_proba = y_proba.max(axis=1)

print("\nConfidence Score Distribution:")
print(f"Mean confidence: {max_proba.mean()*100:.1f}%")
print(f"Median confidence: {np.median(max_proba)*100:.1f}%")
print(f"High confidence (>80%): {(max_proba > 0.8).sum() / len(max_proba) * 100:.1f}%")
print(f"Low confidence (<50%): {(max_proba < 0.5).sum() / len(max_proba) * 100:.1f}%")


Training Performance (Calibrated Model):
                  precision    recall  f1-score   support

 No Side Effects       0.88      0.84      0.86       858
            Mild       0.67      0.93      0.77      1016
        Moderate       0.74      0.58      0.65       614
          Severe       0.86      0.58      0.69       369
Extremely Severe       0.98      0.37      0.53       175

        accuracy                           0.76      3032
       macro avg       0.82      0.66      0.70      3032
    weighted avg       0.78      0.76      0.75      3032


Confidence Score Distribution:
Mean confidence: 48.8%
Median confidence: 45.9%
High confidence (>80%): 4.3%
Low confidence (<50%): 60.2%


In [12]:
# Save improved model and components
os.makedirs('models', exist_ok=True)

joblib.dump(calibrated_model, 'models/severity_model_improved.pkl')
joblib.dump(vectorizer, 'models/tfidf_vectorizer_improved.pkl')
joblib.dump(severity_df.columns.tolist(), 'models/severity_features.pkl')
joblib.dump(severity_map, 'models/severity_map.pkl')

# Save helper functions
model_utils = {
    'extract_severity_features': extract_severity_features,
    'clean_text': clean_text,
    'SEVERITY_KEYWORDS': SEVERITY_KEYWORDS
}
joblib.dump(model_utils, 'models/model_utils.pkl')

print("\nImproved model saved!")
print("Files:")
print("  - models/severity_model_improved.pkl")
print("  - models/tfidf_vectorizer_improved.pkl")
print("  - models/severity_features.pkl")
print("  - models/severity_map.pkl")
print("  - models/model_utils.pkl")


Improved model saved!
Files:
  - models/severity_model_improved.pkl
  - models/tfidf_vectorizer_improved.pkl
  - models/severity_features.pkl
  - models/severity_map.pkl
  - models/model_utils.pkl


In [13]:
# Test with examples to verify better confidence
test_cases = [
    "mild headache that went away after an hour",
    "severe chest pain, extreme difficulty breathing, had to call ambulance",
    "moderate nausea and some dizziness throughout the day",
    "I died and was hospitalized in ICU with organ failure",
    "slight drowsiness in the morning, felt better by noon"
]

print("\nTesting improved model:")
print("="*80)

for text in test_cases:
    # Clean text
    cleaned = clean_text(text)
    
    # Extract features
    text_features = vectorizer.transform([cleaned])
    sev_features = pd.DataFrame([extract_severity_features(text)])
    
    # Combine features
    X_test = hstack([text_features, sev_features.values]).toarray()
    
    # Predict
    pred_class = calibrated_model.predict(X_test)[0]
    proba = calibrated_model.predict_proba(X_test)[0]
    confidence = proba[pred_class] * 100
    
    print(f"\nInput: {text}")
    print(f"Predicted: {severity_map[pred_class]}")
    print(f"Confidence: {confidence:.1f}%")
    print(f"All probabilities: {dict(zip(severity_map.values(), (proba*100).round(1)))}")
    print("-"*80)


Testing improved model:

Input: mild headache that went away after an hour
Predicted: Mild
Confidence: 67.2%
All probabilities: {'No Side Effects': np.float64(13.3), 'Mild': np.float64(67.2), 'Moderate': np.float64(10.1), 'Severe': np.float64(6.9), 'Extremely Severe': np.float64(2.5)}
--------------------------------------------------------------------------------

Input: severe chest pain, extreme difficulty breathing, had to call ambulance
Predicted: Severe
Confidence: 41.2%
All probabilities: {'No Side Effects': np.float64(3.7), 'Mild': np.float64(11.5), 'Moderate': np.float64(27.0), 'Severe': np.float64(41.2), 'Extremely Severe': np.float64(16.7)}
--------------------------------------------------------------------------------

Input: moderate nausea and some dizziness throughout the day
Predicted: Moderate
Confidence: 48.8%
All probabilities: {'No Side Effects': np.float64(2.7), 'Mild': np.float64(33.9), 'Moderate': np.float64(48.8), 'Severe': np.float64(12.9), 'Extremely Severe'